In [1]:
import numpy as np
import datetime
import pandas as pd
from utils.weather_data_util import load_ld_grid_meo_data

### 1. 首先加载使用到的 meo 数据

In [2]:
near_stations = {'BL0': 'london_grid_409',
 'BX1': 'london_grid_472',
 'BX9': 'london_grid_472',
 'CD1': 'london_grid_388',
 'CD9': 'london_grid_409',
 'CR8': 'london_grid_408',
 'CT2': 'london_grid_409',
 'CT3': 'london_grid_409',
 'GB0': 'london_grid_451',
 'GN0': 'london_grid_451',
 'GN3': 'london_grid_451',
 'GR4': 'london_grid_451',
 'GR9': 'london_grid_430',
 'HR1': 'london_grid_368',
 'HV1': 'london_grid_472',
 'KC1': 'london_grid_388',
 'KF1': 'london_grid_388',
 'LH0': 'london_grid_346',
 'LW2': 'london_grid_430',
 'MY7': 'london_grid_388',
 'RB7': 'london_grid_452',
 'ST5': 'london_grid_408',
 'TD5': 'london_grid_366',
 'TH4': 'london_grid_430'}

In [3]:
# bj_meo_stations : meo_station as key and df as value 
bj_grid_meo_dataset, stations, bj_meo_stations = load_ld_grid_meo_data(near_stations)

In [7]:
bj_meo_stations['BX1'].head()

,BX1_temperature,BX1_pressure,BX1_humidity,BX1_wind_direction,BX1_wind_speed/kph
time,,,,,
2017-01-01 00:00:00,5.52,1020.08,92.83,225.18,14.84
2017-01-01 01:00:00,5.64,1018.89,90.95,222.71,16.06
2017-01-01 02:00:00,5.75,1017.69,89.08,220.60,17.31
2017-01-01 03:00:00,5.87,1016.50,87.20,218.77,18.58
2017-01-01 04:00:00,6.00,1015.56,87.46,216.84,19.03


In [8]:
len(bj_meo_stations.keys())

24

In [11]:
print(bj_meo_stations['BX1'].index.min(), bj_meo_stations['BX1'].index.max())

2017-01-01 00:00:00 2018-03-27 05:00:00


### 2. 重复值分析

- 识别重复值的数量，并将重复值去掉
- meo 数据中没有重复数据

In [9]:
for station in bj_meo_stations.keys() :
    
    df = bj_meo_stations[station].copy()
    length = df.shape[0]
    order = range(length)
    df['order'] = pd.Series(order, index=df.index)    
    
    
    df["time"] = df.index
    df.set_index("order", inplace=True)
    
    length_1 = df.shape[0]
    # print("重复值去除之前，共有数据数量", df.shape[0])
    
    used_times = []
    for index in df.index :
        time = df.loc[index]["time"]
        if time not in used_times :
            used_times.append(time)
        else : 
            df.drop([index], inplace=True)

    length_2 = df.shape[0]
    delta = length_1 - length_2
    # print("重复值去除之后，共有数据数量", df.shape[0])
    print("%s 重复数量 : %d" %(station, delta))
    
    df.set_index("time", inplace=True)
    bj_meo_stations[station] = df

RB7 重复数量 : 0
BX9 重复数量 : 0
BL0 重复数量 : 0
GR4 重复数量 : 0
ST5 重复数量 : 0
KF1 重复数量 : 0
GN0 重复数量 : 0
CT3 重复数量 : 0
CD9 重复数量 : 0
CR8 重复数量 : 0
CD1 重复数量 : 0
GN3 重复数量 : 0
CT2 重复数量 : 0
MY7 重复数量 : 0
TD5 重复数量 : 0
GR9 重复数量 : 0
KC1 重复数量 : 0
HV1 重复数量 : 0
GB0 重复数量 : 0
BX1 重复数量 : 0
TH4 重复数量 : 0
LH0 重复数量 : 0
LW2 重复数量 : 0
HR1 重复数量 : 0


### 3. 缺失值分析

#### 3.1 局部缺失

判断有没有局部缺失：没有局部缺失！

In [12]:
for station in bj_meo_stations.keys():
    df = bj_meo_stations[station]
    print(station, pd.isnull(df).any().any())

RB7 False
BX9 False
BL0 False
GR4 False
ST5 False
KF1 False
GN0 False
CT3 False
CD9 False
CR8 False
CD1 False
GN3 False
CT2 False
MY7 False
TD5 False
GR9 False
KC1 False
HV1 False
GB0 False
BX1 False
TH4 False
LH0 False
LW2 False
HR1 False


#### 3.2 整体缺失

统计缺失小时的连续值
- 如果一个缺失小时在一个长度小于等于5小时的缺失时段内，就进行补全
- 如果超过5小时，就舍弃，将全部值补成 NAN，**这也是整个表中唯一可能出现 NAN 的情况**

In [13]:
for station in bj_meo_stations.keys() :
    df = bj_meo_stations[station].copy()
    
    min_time = df.index.min()
    max_time = df.index.max()

    min_time = datetime.datetime.strptime(min_time, '%Y-%m-%d %H:%M:%S')
    max_time = datetime.datetime.strptime(max_time, '%Y-%m-%d %H:%M:%S')
    delta_all = max_time - min_time
    
    all_length = delta_all.total_seconds()/3600 + 1
    real_length = df.shape[0]
    # print("在空气质量数据时间段内，总共应该有 %d 个小时节点。" %(all_length))
    # print("实际的时间节点数是 ", real_length)
    print("%s 缺失时间节点数量是 %d" %(station, all_length-real_length))

RB7 缺失时间节点数量是 0
BX9 缺失时间节点数量是 0
BL0 缺失时间节点数量是 0
GR4 缺失时间节点数量是 0
ST5 缺失时间节点数量是 0
KF1 缺失时间节点数量是 0
GN0 缺失时间节点数量是 0
CT3 缺失时间节点数量是 0
CD9 缺失时间节点数量是 0
CR8 缺失时间节点数量是 0
CD1 缺失时间节点数量是 0
GN3 缺失时间节点数量是 0
CT2 缺失时间节点数量是 0
MY7 缺失时间节点数量是 0
TD5 缺失时间节点数量是 0
GR9 缺失时间节点数量是 0
KC1 缺失时间节点数量是 0
HV1 缺失时间节点数量是 0
GB0 缺失时间节点数量是 0
BX1 缺失时间节点数量是 0
TH4 缺失时间节点数量是 0
LH0 缺失时间节点数量是 0
LW2 缺失时间节点数量是 0
HR1 缺失时间节点数量是 0


#### 3.3 整体缺失补充

In [14]:
for station in bj_meo_stations.keys() :
    df = bj_meo_stations[station].copy()
    print(station, df.shape)

RB7 (10806, 5)
BX9 (10806, 5)
BL0 (10806, 5)
GR4 (10806, 5)
ST5 (10806, 5)
KF1 (10806, 5)
GN0 (10806, 5)
CT3 (10806, 5)
CD9 (10806, 5)
CR8 (10806, 5)
CD1 (10806, 5)
GN3 (10806, 5)
CT2 (10806, 5)
MY7 (10806, 5)
TD5 (10806, 5)
GR9 (10806, 5)
KC1 (10806, 5)
HV1 (10806, 5)
GB0 (10806, 5)
BX1 (10806, 5)
TH4 (10806, 5)
LH0 (10806, 5)
LW2 (10806, 5)
HR1 (10806, 5)


In [15]:
delta = datetime.timedelta(hours=1)

for station in bj_meo_stations.keys() :
    df = bj_meo_stations[station].copy()
    nan_series = pd.Series({key:np.nan for key in df.columns})
    
    min_time = df.index.min()
    max_time = df.index.max()

    min_time = datetime.datetime.strptime(min_time, '%Y-%m-%d %H:%M:%S')
    max_time = datetime.datetime.strptime(max_time, '%Y-%m-%d %H:%M:%S')

    time = min_time
    
    while time <=  max_time :
        
        time_str = datetime.date.strftime(time, '%Y-%m-%d %H:%M:%S')
        if time_str not in df.index :
            
            # 前边第几个是非空的
            found_for = False
            i = 0
            while not found_for :
                i += 1
                for_time = time - i * delta
                for_time_str = datetime.date.strftime(for_time, '%Y-%m-%d %H:%M:%S')
                if for_time_str in df.index :
                    for_row = df.loc[for_time_str]
                    for_step = i
                    found_for = True

            # 后边第几个是非空的
            found_back = False
            j = 0
            while not found_back :
                j += 1
                back_time = time + j * delta
                back_time_str = datetime.date.strftime(back_time, '%Y-%m-%d %H:%M:%S')
                if back_time_str in df.index :
                    back_row = df.loc[back_time_str]
                    back_step = j
                    found_back = True
        
            all_steps = for_step + back_step
        
            if all_steps <= 5 :
                delata_values = back_row - for_row
                df.loc[time_str] = for_row + (for_step/all_steps) * delata_values
            else :
                df.loc[time_str] = nan_series
        
        time += delta
    bj_meo_stations[station] = df
    
    print("%s : length of data is %d" %(station, df.shape[0]))

RB7 : length of data is 10806
BX9 : length of data is 10806
BL0 : length of data is 10806
GR4 : length of data is 10806
ST5 : length of data is 10806
KF1 : length of data is 10806
GN0 : length of data is 10806
CT3 : length of data is 10806
CD9 : length of data is 10806
CR8 : length of data is 10806
CD1 : length of data is 10806
GN3 : length of data is 10806
CT2 : length of data is 10806
MY7 : length of data is 10806
TD5 : length of data is 10806
GR9 : length of data is 10806
KC1 : length of data is 10806
HV1 : length of data is 10806
GB0 : length of data is 10806
BX1 : length of data is 10806
TH4 : length of data is 10806
LH0 : length of data is 10806
LW2 : length of data is 10806
HR1 : length of data is 10806


#### 3.4 风向缺失值处理
- 暂时使用 0 替换缺失的风向

In [16]:
for station in bj_meo_stations.keys():
    df = bj_meo_stations[station].copy()
    df.replace(999017,0, inplace=True)
    bj_meo_stations[station] = df

#### 3.5 拼成整表，并保存

In [17]:
bj_meo_stations_merged = pd.concat(list(bj_meo_stations.values()), axis=1)

In [23]:
stations_to_predict = ['BL0','CD1','CD9','GN0','GN3','GR4','GR9','HV1','KF1','LW2','MY7','ST5','TH4']

In [26]:
all_features = []
for column_name in bj_meo_stations_merged.columns : 
    for station in stations_to_predict :
        if station in column_name :
            all_features.append(column_name)

In [29]:
bj_meo_stations_merged = bj_meo_stations_merged[all_features]

In [30]:
bj_meo_stations_merged.shape

(10806, 65)

In [19]:
bj_meo_stations_merged.sort_index(inplace=True)

In [20]:
bj_meo_stations_merged.to_csv("test/ld_meo_data.csv")

需要将 `ld_meo_data.csv`左上角位置的空格补上 "time" 

### 4 数据归一化

In [21]:
describe = bj_meo_stations_merged.describe()
describe.to_csv("test/ld_meo_describe.csv")

In [22]:
df_norm = (bj_meo_stations_merged - describe.loc['mean']) / describe.loc['std']
df_norm.to_csv("test/ld_meo_norm_data.csv")